In [ ]:
import ase
import networkx as nx
import os
from monty.serialization import loadfn
from glob import glob
import time
from tqdm import tqdm
import collections
import numpy as np
import matplotlib.pyplot as plt
import ast
import h5py


from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN, CovalentBondNN
from pymatgen.util.graph_hashing import weisfeiler_lehman_graph_hash

from radqm9_pipeline.elements import read_elements
from radqm9_pipeline.modules import merge_data, flatten

# Load Data

In [ ]:
elements_dict = read_elements('/pscratch/sd/m/mavaylon/sam_ldrd/radqm9_pipeline/src/radqm9_pipeline/modules/elements.pkl')

In [1]:
base_dir = "/pscratch/sd/m/mavaylon/radqm9pipeline_cut_paste/update_20240123/Trajectories/trajectories"

In [ ]:
merged_data=merge_data(base_dir)

# Resolve Split Trjacetories

In [2]:
def flatten_filter(data: dict):
    """
    Flatten bucket (dict) to list
    """
    data_to_be_parsed = []
    for mol_id in tqdm(data):
        for pair in data[mol_id]:
            data_to_be_parsed.append(pair)
    return data_to_be_parsed    

In [ ]:
def bucket_mol_id(data: list):
    """
    Return: 
    - Good_data--> This means data that either had no duplicates or data that has been able to be mended 
    - bad_ids --> are events that are not able to be mended
    - length is the total number of events both mended and unmended
    
    TODO: fix the merge duplication in geometries, for now this does not matter since we downsample.
  
    """
    bucket = {}
    
    # Step 1    
    """
    Bucket into mol ids
    """
    for event in tqdm(data):
        try:
            bucket[event['mol_id']].append(event) 
        except KeyError:
            bucket[event['mol_id']] = [event]
    
    # Step 2
    """
    Find duplicate pairs in each mol id.
    What are duplicate pairs? These are training sessions that most likely continued as a separate job. 
    They need to be attached if possible. 
    """
    length=0
    bad_ids = []
    for mol_id in tqdm(bucket):
        pairs = [event['charge_spin'] for event in bucket[mol_id]]
        # Get a list of all charge_spins that have duplicates.
        duplicate_pairs = [item for item, count in collections.Counter(pairs).items() if count > 1] 
        if len(duplicate_pairs)!=0:
            """
            Handle the duplicate pairs to see if they can be merged
            """
            len_p = len(bucket[mol_id])
            for dup in duplicate_pairs:
                bad_data = []
                # Order events
                case_events = [event for event in bucket[mol_id] if event['charge_spin']==dup]
                bad_events = []
                bad_events += case_events

                for event in case_events: # remove events to be fixed. If fixable, add back at the end.
                    bucket[mol_id].remove(event) 
        
                ordered = [case_events[0]]
                del case_events[0]
                
                counter = 0
                threshold = 30
                while len(case_events)!=0:
                    if len(bad_data)==0:
                        for event in case_events:
                            beg = event['geometries'][0]
                            end = event['geometries'][len(event['geometries'])-1]

                            ordered_beg = ordered[0]['geometries'][0]
                            ordered_end = ordered[len(ordered)-1]['geometries'][len(ordered[len(ordered)-1]['geometries'])-1]

                            if beg==ordered_end:
                                ordered.append(event)
                                case_events.remove(event)
                            elif end==ordered_beg:
                                ordered.insert(0, event)
                                case_events.remove(event)
                            else:
                                counter+=1
                                if counter>threshold:
                                    bad_data.append(mol_id)
                                else:
                                    continue
                    else:
                        break
                    
                if len(bad_data)==0:                
                    # Merge the ordered events: forces, geometries
                    merged_event = {}
                    merged_event['task_id'] = ordered[0]['task_id']
                    merged_event['mol_id'] = mol_id
                    merged_event['name'] = ordered[0]['name']
                    merged_event['charge'] = ordered[0]['charge']
                    merged_event['spin'] = ordered[0]['spin']
                    merged_event['charge_spin'] = ordered[0]['charge_spin']
                    merged_event['species'] = ordered[0]['species']

                    geometries = []
                    energies = []
                    grads = []
                    mulliken = []
                    resp = []
                    dipole_moments = []
                    dipole_moments_resp = []
                    for event in ordered:
                        geometries += event['geometries']
                        energies += event['energies']
                        grads += event['gradients']
                        mulliken += event['mulliken']
                        resp += event['resp']
                        dipole_moments += event['dipole_moments']
                        dipole_moments_resp += event['dipole_moments_resp']

                    merged_event['geometries'] = geometries
                    merged_event['energies'] = energies
                    merged_event['gradients'] = grads
                    merged_event['mulliken'] = mulliken
                    merged_event['resp'] = resp
                    merged_event['dipole_moments'] = dipole_moments
                    merged_event['dipole_moments_resp'] = dipole_moments_resp

                    bucket[mol_id].append(merged_event)
                else:
                    bad_ids += bad_events
            len_r = len(bucket[mol_id])
            length += len_p-len_r
    good_data = flatten_filter(bucket)
        
    return good_data, bad_ids, length

In [ ]:
g_data_bucket, b_data_bucket, length = bucket_mol_id(merged_data)

In [ ]:
def add_unique_id(data: list):
    for item in data:
        item['mol_cs'] = str(item['mol_id']) + str(item['charge_spin'])

In [ ]:
add_unique_id(g_data_bucket)

# Filter Forces

In [ ]:
def force_magnitude_filter(cutoff: float,
                           data: list):
    """
    This method returns both data that meets the cuttoff value and data that is equal to or above the cuttoff value.
    If this is run before downsampling, it removes the entire data point trajectory.
    
    Returns: lists
    """
    good = []
    bad = []
    
    for item in tqdm(data):
        forces = item['gradients']
        for path_point in forces:
            next_item = False
            for atom in path_point:
                res = np.sqrt(sum([i**2 for i in atom]))
                if res >= cutoff:
                    bad.append(item)
                    next_item = True
                    break
            if next_item:
                break
        if not next_item:
            good.append(item)
                            
    return good, bad

In [3]:
g_data, b_data = force_magnitude_filter(cutoff=10.0, data=g_data_bucket)

NameError: name 'force_magnitude_filter' is not defined

# Sparse Data

In [ ]:
def average_force_trajectory(pair):
    """
    This method will take a specfic spin charge pair. At each point in the optimization trajectory, the 
    """
    forces = {}
    for i in range(len(pair['gradients'])):
        temp = []
        for atom in pair['gradients'][i]:
            res = np.sqrt(sum([j**2 for j in atom]))
            temp.append(res)
        forces[i] = np.mean(temp)
    del forces[0]
    return forces

In [ ]:
def sparse_trajectory(bucket: list):
    """
    This takes the cleaned data and will sparsifiy the optimization trajectories. How this is done will depend on the
    charge_spin pair:
    - Neutral Singlet (0,1): First and Last
    - Other: First, Last, and structure with the highest molecular force other than the First.
    
    Note: Molecular Force is just the average of the force magnitudes of each atom in the molecule:
    """
    
    for pair in tqdm(bucket):
        if pair['charge_spin'] == '0,1':
            geometries = [pair['geometries'][0], pair['geometries'][-1]]
            energies = [pair['energies'][0], pair['energies'][-1]]
            grads = [pair['gradients'][0], pair['gradients'][-1]]
            mulliken = [pair['mulliken'][0], pair['mulliken'][-1]]
            resp = [pair['resp'][0], pair['resp'][-1]]
            dipole_moments = [pair['dipole_moments'][0], pair['dipole_moments'][-1]]
            dipole_moments_resp = [pair['dipole_moments_resp'][0], pair['dipole_moments_resp'][-1]]

            pair['geometries'] = geometries
            pair['energies'] = energies
            pair['gradients'] = grads
            pair['mulliken'] = mulliken
            pair['resp'] = resp
            pair['dipole_moments'] = dipole_moments
            pair['dipole_moments_resp'] = dipole_moments_resp
        else:
            force_dict = average_force_trajectory(pair)
            max_index = max(force_dict, key=force_dict.get)

            geometries = [pair['geometries'][0], pair['geometries'][max_index], pair['geometries'][-1]]
            energies = [pair['energies'][0], pair['energies'][max_index], pair['energies'][-1]]
            grads = [pair['gradients'][0], pair['gradients'][max_index], pair['gradients'][-1]]
            mulliken = [pair['mulliken'][0], pair['mulliken'][max_index], pair['mulliken'][-1]]
            resp = [pair['resp'][0], pair['resp'][max_index], pair['resp'][-1]]
            dipole_moments = [pair['dipole_moments'][0], pair['dipole_moments'][max_index], pair['dipole_moments'][-1]]
            dipole_moments_resp = [pair['dipole_moments_resp'][0], pair['dipole_moments_resp'][max_index], pair['dipole_moments_resp'][-1]]

            pair['geometries'] = geometries
            pair['energies'] = energies
            pair['gradients'] = grads
            pair['mulliken'] = mulliken
            pair['resp'] = resp
            pair['dipole_moments'] = dipole_moments
            pair['dipole_moments_resp'] = dipole_moments_resp

In [ ]:
def build_graph(species, position):
    atoms = ase.atoms.Atoms(symbols=species,
                            positions=position)
    mol = AseAtomsAdaptor.get_molecule(atoms)
    graph = MoleculeGraph.with_local_env_strategy(mol, OpenBabelNN())
    return graph

In [ ]:
def filter_broken_graphs(data: list):
    broken = []
    good = []
    
    for item in tqdm(data):
        for traj_point in item['geometries'][1:]: # Ignore the first is in the filter_broken_graphs
            graph = build_graph(item['species'], traj_point)
            connected = nx.is_connected(graph.graph.to_undirected())
            if not connected:
                broken.append(item)
                break
        if connected:
            good.append(item)

    return good, broken


In [ ]:
filtered_good, broken =filter_broken_graphs(g_data)

### Check point clean data

In [ ]:
import h5py
with h5py.File('/pscratch/sd/m/mavaylon/new_pipe/redone_2024_5_10_full_filtered.h5', 'w') as file:
    g1 =file.create_group('clean_data')
    raw=[]
    for item in tqdm(filtered_good):
        raw.append(str(item))
    g1.create_dataset('data',data=raw)

### Read Clean Data

In [ ]:
read_file=h5py.File('/pscratch/sd/m/mavaylon/new_pipe/redone_2024_5_10_full_filtered.h5', 'r')

In [ ]:
merged_all_clean_data=[]
for point in tqdm(read_file['clean_data']['data']):
    point = ast.literal_eval(point.decode('utf-8'))
    merged_all_clean_data.append(point)

# Split Data

In [ ]:
def get_molecule_weight(data: list):
    """
    The method takes in a list of data (either trajectories or single points) and sorts into a distribution
    dictionary. The keys are the species/formula and the value of each key is the weight. appearing number of times the species
    appears in the dataset.
    """
    dict_dist = {}
    for item in tqdm(data):
        species_num = []
        species=''.join((sorted(item['species'])))
        
        for element in item['species']:
            species_num.append(elements_dict[element])

        species_sum = sum(species_num)
        try:
            dict_dist[species].append(species_sum)
            dict_dist[species] = [dict_dist[species][0]]*len(dict_dist[species])
        except KeyError:
            dict_dist[species] = [species_sum]
        
    return dict_dist

In [4]:
def molecule_weight(data: list, weight_dict):
    """
    This method takes in data and assigns the mass.
    Python does a weird thing floats e.g., {126.15499999999993, 126.15499999999994}, having this and
    get_molecule_weight gurantees that species that are the same are not being assigned different weights.
    """
    for item in tqdm(data):
        weight = weight_dict[''.join((sorted(item['species'])))][0]
        item['molecule_mass'] = weight

In [ ]:
merged_dist = get_molecule_weight(cleaned)
molecule_weight(cleaned, merged_dist)

In [ ]:
def weight_to_data(data: list):
    """
    This method buckets the data by the mass such that the dict key is the mass and the values are the data
    points.
    """
    dict_data = {}
    for item in tqdm(data):
        try:
            dict_data[item['molecule_mass']].append(item)
        except KeyError:
            dict_data[item['molecule_mass']] = [item]
    return dict_data

In [ ]:
wtd = weight_to_data(cleaned)

In [ ]:
def length_dict(data: dict):
    """
    This method takes in the output of weight_to_data and returns a dictionary that is sorted from largest
    to smallest mass. The keys are the mass and the values are the number of appearances.
    """
    length_dict = {key: len(value) for key, value in data.items()}
    sorted_length_dict = {k: length_dict[k] for k in sorted(length_dict, reverse=True)}
    
    return sorted_length_dict

In [ ]:
sld = length_dict(wtd)

In [ ]:
"""
TODO: make into a function

The split method is as follows:
I have a dictionary that is sorted from highest mass to lowest mass. The value of each key is the number of times
that mass is in the data. Another way to think of this, is the number of trajectories or SPs that have that
mass. 

We have a list for each split that stores the the masses.
We have three variables that store the size of the splits. 

Each iteration will add a mass to a split. This ensures that the mass is in one split. This means that that species
is only in that split. 

It will continue to add until all the masses have been added. 
"""

# Take initial points (the highest masses) and have them in the data
train_mass = [152.037]
test_mass = [144.09200000000007]
val_mass = [143.1080000000001]

train = sld[152.037] # trackers for dataset sizes
test = sld[144.09200000000007]
val = sld[143.1080000000001]

sld.pop(152.037)
sld.pop(144.09200000000007)
sld.pop(143.1080000000001)

# Sort the data 
# data is a dict: mass-># of trajs
for mass in sorted_length_dict:
    temp_total = train+val+test
    train_ratio = .65-(train/temp_total)
    test_ratio = .25-(test/temp_total)
    val_ratio = .1-(val/temp_total)
    
    if train_ratio > val_ratio and train_ratio>test_ratio:
        train_mass.append(mass)
        train += sorted_length_dict[mass]
    if val_ratio > train_ratio and val_ratio>test_ratio:
        val_mass.append(mass)
        val += sorted_length_dict[mass]
    if test_ratio > val_ratio and test_ratio>train_ratio:
        test_mass.append(mass)
        test += sorted_length_dict[mass]

In [ ]:
# sanity check
train/(train+val+test)

In [5]:
val/(train+val+test)

NameError: name 'val' is not defined

In [ ]:
test/(train+val+test)

### Visualize the splits

In [ ]:
sld = length_dict(wtd) # you need to call this again yes

In [ ]:
train_subset={key: sorted_length_dict[key] for key in train_mass if key in sorted_length_dict}
test_subset={key: sorted_length_dict[key] for key in test_mass if key in sorted_length_dict}
val_subset={key: sorted_length_dict[key] for key in val_mass if key in sorted_length_dict}

In [ ]:
train_temp=[[x]*train_subset[x] for x in train_subset]
test_temp=[[x]*test_subset[x] for x in test_subset]
val_temp=[[x]*val_subset[x] for x in val_subset]


In [ ]:
from itertools import chain

train_subset_merged = list(chain.from_iterable(train_temp))
test_subset_merged = list(chain.from_iterable(test_temp))
val_subset_merged = list(chain.from_iterable(val_temp))


In [ ]:
plt.hist(train_subset_merged, bins=50)
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.xlabel('Molecule Mass')
plt.title('Train')

In [ ]:
plt.hist(test_subset_merged, bins=50)
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.xlabel('Molecule Mass')
plt.title('Test')

In [ ]:
plt.hist(val_subset_merged, bins=50)
plt.ylabel('Frequency (log)')
plt.yscale('log')
plt.xlabel('Molecule Mass')
plt.title('Val')

### Make Manual Switches

In [ ]:
switch=[45.04100000000001,
54.09200000000001,
60.05600000000001,
70.05099999999999,
72.10699999999997,
76.05799999999999,
78.06999999999998,
84.08199999999998,
85.06599999999997,
87.08199999999998,
88.06199999999998,
93.08899999999998,
95.10499999999999,
97.11699999999996,
98.06099999999999,
102.09299999999996,
102.17699999999992,
107.06799999999998]

In [ ]:
for mass in switch:
    val_mass.append(mass)
    val += sorted_length_dict[mass]
    
    test_mass.remove(mass)
    test -= sorted_length_dict[mass]

This is is to ensure even mass disitrubtion across the sets. 

# Resolve splits to data

In [ ]:
train_data = [wtd[x] for x in train_mass]
train_data = list(chain.from_iterable(train_data))

val_data = [wtd[x] for x in val_mass]
val_data = list(chain.from_iterable(val_data))

test_data = [wtd[x] for x in test_mass]
test_data = list(chain.from_iterable(test_data))


In [ ]:
data = {'train':train_data,
        'val': val_data,
        'test': test_data}

In [ ]:
def build_minimal_atoms(data: dict,
                energy: str = None,
                forces: str = None,
                charge:str = None,
                spin:str = None,
                train = False) -> ase.Atoms:
    """ 
    Populate Atoms class with atoms in molecule.
        atoms.info : global variables
        atoms.array : variables for individual atoms
        
    Both "energy" and "forces" are the dict strings in data.
    """
    atom_list = []
    for i in range(len(data['geometries'])):
        atoms = ase.atoms.Atoms(
            symbols=data['species'],
            positions=data['geometries'][i]
        )
        if energy is not None:
            atoms.info['energy'] = data[energy][i]
        if forces is not None:
            atoms.arrays['forces'] = np.array(data[forces][i])
        if charge is not None:
             atoms.info['charge'] = data[charge]
        if spin is not None:
            atoms.info['spin'] = data[spin]

        if i == 0:
            atoms.info['position_type'] = 'start'
        if i == 1:
            if data['charge_spin'] == '0,1':
                atoms.info['position_type'] = 'end'
            else:
                atoms.info['position_type'] = 'middle'
        if i == 2:
            atoms.info['position_type'] = 'end'
        atom_list.append(atoms)
    return atom_list


In [ ]:
def build_minimal_atoms_iterator(data: list,
                         train=False):
    """
    This method assumes the data has been validated. This will create ASE atoms to be written.
    
    The input needs to be a list of lists that contain the event dictionaries. Each inner list needs to represent all the events for a single
    mol_id.
    """
    data_set=[]
    for point in tqdm(data):
        atoms=build_minimal_atoms(point, energy='energies', forces='gradients', charge='charge', spin='spin', train=train)
        data_set+=atoms
    return data_set

In [ ]:
build = {}
for split in data:
    if split == 'train':
        build[split] = build_minimal_atoms_iterator(data[split], train=True)
    else:
        build[split] = build_minimal_atoms_iterator(data[split])

In [ ]:
def create_dataset(data: dict,
                   file_name:str,
                   path:str):
    """
    This method will handle the I/O for writing the data to xyz files to the path provided.
    """
    train_data = data['train']
    val_data = data['val']
    test_data = data['test']
    
    train_file = os.path.join(path,file_name+'_train.xyz')
    ase.io.write(train_file, train_data,format="extxyz")
     
    val_file = os.path.join(path,file_name+'_val.xyz')
    ase.io.write(val_file, val_data,format="extxyz")
    
    test_file = os.path.join(path,file_name+'_test.xyz')
    ase.io.write(test_file, test_data,format="extxyz")

In [ ]:
full_data_path ='/pscratch/sd/m/mavaylon/chem_final_data/Traj/Full_data/'

In [ ]:
charge_data_path ='/pscratch/sd/m/mavaylon/chem_final_data/Traj/Charge_data/'